### Code: Berezinskii-Kosterlitz-Thouless Renormalization Group Flow at a Quantum Phase Transition
Matthias Thamm, Harini Radhakrishnan, Hatem Barghathi, Chris Herdman, Arpan Biswas, Bernd Rosenow, and Adrian Del Maestro 


#### Abstract 
We present a controlled numerical study of the Berezinskii-Kosterlitz-Thouless (BKT) transition in the one-dimensional Bose-Hubbard model at unit filling, providing evidence of the characteristic logarithmic finite-size scaling of the BKT transition.nEmploying density matrix renormalization group and quantum Monte Carlo simulations under periodic boundary conditions, together with a systematic finite-size scaling analysis of bipartite particle number fluctuations, we resolve boundary-induced complications that previously obscured critical scaling. We demonstrate that a suitably chosen central region under open boundaries reproduces universal RG signatures, reconciling earlier discrepancies. Finally, leveraging a non-parametric Bayesian analysis, we determine the critical interaction strength with high precision, establishing a benchmark for BKT physics in one-dimensional quantum models.

#### Requirements 
To run this code, Julia version 1.10.4 or higher is required. The cell below will install the required packages.

### Install packages

In [ ]:
using Pkg 
Pkg.activate(".")

In [ ]:
Pkg.add([
    "Plots", 
    "PyFormattedStrings", 
    "NonlinearSolve", 
    "StaticArrays", 
    "Printf", 
    "Integrals", 
    "FastClosures", 
    "LaTeXStrings", 
    "DataFrames",  
    "NPZ", 
    "Measures", 
    "PyCall"]);

### Import

In [ ]:
using Plots
using PyFormattedStrings
using NonlinearSolve
using StaticArrays
using Printf
using Integrals
using FastClosures
using LaTeXStrings
using DataFrames 
using NPZ
using Measures
using PyCall
using DelimitedFiles

### Include 

In [ ]:
include("./plot_utils.jl"); 
include("./load.jl");
include("./fit.jl");
include("./rg.jl");
include("./fss.jl")
include("./plot_routines.jl");

### Figure 1: BKT RG Flow
RG flow close to the BKT phase transition constructed from the numerical DMRG simulations for periodic boundary conditions via 

\begin{align*}
    2\ln\!\left(\frac{L_2}{L_1}\right)  &= \int_{K_2}^{K_1} \frac{{\rm d}K}{K^2 (\ln(K/2)-\zeta) + 2K} \\
            \zeta  &\equiv \frac{2}{K} + \ln\left(\frac{K}{2}\right) - \frac{\pi^2}{2} \left(\frac{V}{E_r}\right)^2  \ .
\end{align*}

Here, $K$ is the Luttinger parameter, $V$ the strength of the periodic lattice potential, $E_r=u\pi/a$ a characteristic energy scale, and $U$ the interacting strength of the Bose-Hubbard model

\begin{align*}
    H &= -J \sum_{i=1}^L \left( b_i^\dagger b^{\phantom{\dagger}}_{i+1}+ b_{i+1}^\dagger b^{\phantom{\dagger}}_i \right) + \frac{U}{2} \sum_{i=1}^L n_i (n_i-1) 
\end{align*}

considered in the numerical simulations. The dashed black line is the separatrix marking the critical line of the phase transition with $\zeta(U_c/J)=1$. To the right of the separatrix, the system is in the superfluid phase (SF),  and to the left  it is in the Mott insulating phase (MI). Each colored line corresponds to one value of $\zeta(U/J)$ obtained from the numerical results for the largest two system sizes for each $U/J$.  Circles show the points for individual system sizes (larger circles correspond to larger $L$, $L\in\{16,32,48,64,96,128\}$).  Open circles are obtained by fits of the $L\to\infty$ limit of 

\begin{align*}
    \mathcal{F}_{\rm{pbc}}(\ell; K, \alpha) &= 
    &= \frac{K}{2\pi^2}\ln{\left[1+\frac{\sin^2{(\pi\ell/L)}}{\sinh^2{(\pi\alpha/L)}}\right]}  
\end{align*}

to data obtained for infinite boundary conditions using the VUMPS algorithm [V. Zauner-Stauber, L. Vanderstraeten, M. T. Fishman, F. Verstraete, and J. Haegeman, Physical Review B 97,
045145 (2018).]. Crosses are obtained from QMC.  We find excellent agreement of the DMRG data with the predicted RG flow and between the different methods.

In [ ]:
begin  
    # color lookup
	Us = get_values_of_U(;path="../data/pbc/DMRG/")
	local col_lookup = get_color_lookup(Us)
	# plot options
	local l = grid(1, 2, widths=[0.8,0.2])
    local ms_lookup =  get_marker_size_lookup()
  
	local p1 = plot(; xlabel=L"$K$", ylabel=L"$V/E_r$", ylim=(-0.00015,0.06366197723675814), xlim=(1.645,2.0+.77), legend=nothing,foreground_color_legend =nothing, legendtitle = nothing, legendtitlefontsize=9, handlelength=8  ) 
     
	# pbc dmrg
	local Us, ζs, ζserr = plot_dmrg_rg_flow!(p1, col_lookup, ms_lookup )  
	# avoid accidental overwrite for values required below
	__Us, __ζs, __ζserr = Us, ζs, ζserr
	# separatrix 
    local Ks_sep, Vs_sep = get_flow_curve(1.0)
    plot!(p1, Ks_sep[Ks_sep.>=2.0], Vs_sep[Ks_sep.>=2.0]; color=:black, linewidth=1.0, linestyle=:dash, label=nothing) 
	add_arrow!(p1,Ks_sep[Ks_sep.>2.0], Vs_sep[Ks_sep.>2.0], 0.28*2/(4pi) ,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.>2.0], Vs_sep[Ks_sep.>2.0], 0.35*2/(4pi) ,:black;debug_print=true)
	plot!(p1, Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0]; color=:black, linewidth=1.0, linestyle=:dot, label=nothing)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.1*2/(4pi) ,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.2*2/(4pi) ,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.3*2/(4pi) ,:black;debug_print=true)

	# vumps data 
	plot_vumps_rg_flow!(p1, col_lookup, ms_lookup )

	# qmc data 
	plot_qmc_rg_flow!(p1, col_lookup, ms_lookup )
	
	# manually draw legend 
	local p_leg = plot(; framestyle = nothing) 
	draw_legend!(p_leg, Us, ζs, col_lookup, ms_lookup, highlight_U = [3.275])	 
	
	# add phase labels
	annotate!(p1, 2.0, 0.38*2/(4pi) , text(L"\textrm{MI}", 10, :vcenter, :hcenter))
	annotate!(p1, 2.625, 0.38*2/(4pi) , text(L"\textrm{SF}", 10, :vcenter, :hcenter)) 
	
	
	local p = plot(p1, p_leg, layout = l, size=(1.2*600,1.1*600/2), bottom_margin=[2mm 2mm], left_margin=[2mm -5mm] )  
	savefig(p,"../figures/001_BKT_RG_flow_Bose-Hubbard.pdf");
	savefig(p,"../figures/001_BKT_RG_flow_Bose-Hubbard.png");
	#p 
end

#### Write flow data to file

Save data close to critical point to file for GP BO analysis.

In [ ]:
one_minus_ζ2 = (1 .- __ζs).^2
done_minus_ζ2 =  2*abs.( 1 .- __ζs) .* __ζserr

idxs = 3.1 .<=  __Us .<= 3.4 
open("../data/flow/flow_dmrg_pbc_data.txt","w") do f 
    write(f, "# U        zeta                 error_zeta             (1-zeta)^2               error \n")
    writedlm(f, [__Us[idxs] __ζs[idxs] __ζserr[idxs] one_minus_ζ2[idxs] done_minus_ζ2[idxs]], header=true)
end

### Figure 2: Finite Size Scaling
Finite size scaling of the Luttinger parameter as a function of the system size $L$ according to the RG flow. The circles depict the results obtained from high precision DMRG calculations with open boundary conditions. Dashed lines show fits of the corresponding scaling form to the data. We show the three scaling regimes (i) in the superfluid phase (upper orange curve) according to 

\begin{align*}
K_{\rm SF} &\sim K^* + \kappa_0 \left(\frac{L_0}{L}\right)^{2(K^*-2)} \ ,
\end{align*}

(ii) for the value of $U$ closest to the separatrix (center red curve) using 

\begin{align*}
 K_{\rm sep} &\sim 2  + \frac{1}{1/\kappa_0 + \ln L/L_0} \ ,
\end{align*}

and (iii) in the insulating phase (bottom blue curve) according to 

\begin{align*}
 K_{\rm MI} &\sim 1 + (1+\kappa_0)\left(\frac{L_0}{L}\right)^{8 (1-\zeta)}\, .
\end{align*}

Squares depict data obtained for open boundary conditions using a finetuned fitting interval for extraction of $K$ (see Supplement).

In [ ]:
# requires to run the cell above first 
ζlookup = Dict{Float64,Float64}()
for (U,ζ) in zip(__Us,__ζs)
    ζlookup[U] = ζ
end 

In [ ]:
begin 
	local Us = [3.2,3.275,3.3]  
	local Ls_pbc = [16,32,48,64,96,128]
    local Ls_obc = [200,256,330,460,512]  
	local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/"))
	local cols = [col_lookup[U] for U in Us]

	local xlim = (0.0,0.375)
    local ylim = (1.95001,2.27)
    
    
	local boff = 0mm #-7mm
    local p = plot(;ylabel=L"$K$",xlabel=L"$1/\ln(L)$",xlim=xlim,ylim=ylim, size=(1.2*300,1.0*250),bottom_margin=[boff boff 0mm]) #,yticks=([1.6,1.8,2.0],["1.60","1.80","2.00"]))

    # labels
    annotate!(p,0.172+0.003,2.25, text(latexstring("\\mathrm{OBC}"),:center,9))
    annotate!(p,0.2305+0.003,2.25, text(latexstring("\\mathrm{PBC}"),:center,9))
    
	local xoff=0.003
    local x_right = 0.26
    
    annotate!(p,x_right,2.06,text(latexstring(f"$U/J\\ \\ \\ \\ \\ \\ \\ \\zeta$"),9,:left )) 
    plot!(p,[x_right-0.015,x_right-0.005], [2.03,2.03];color= cols[1])
	annotate!(p,x_right,2.03,text(latexstring(f"${Us[1]:2.1f}\\ \\ \\ \\ \\ {ζlookup[Us[1]]:2.4f}$"),9,:left )) 
    plot!(p,[x_right-0.015,x_right-0.005], [2.00,2.00];color= cols[2])
    annotate!(p,x_right,2.00,text(latexstring(f"${Us[2]:2.3f}\\;\\  {ζlookup[Us[2]]:2.4f}$"),9,:left ))   
    plot!(p,[x_right-0.015,x_right-0.005], [1.97,1.97];color= cols[3])
    annotate!(p,x_right,1.97,text(latexstring(f"${Us[3]:2.1f}\\ \\ \\ \\  \\ {ζlookup[Us[3]]:2.4f}$"),9,:left ))

	# x-axis transformation: plot 1/log(L)
	x_trafo = (x) -> 1.0 ./ log.(x)

	for (U, (i_c,c)) in zip(Us,enumerate(cols))
        #PBC
        
		local Ks_lin = zeros(Float64,length(Ls_pbc))
		local Ks_lin_err = zeros(Float64,length(Ls_pbc))
		local as_lin = zeros(Float64,length(Ls_pbc))
		for (i,L) in enumerate(Ls_pbc)
		    local ls, Fs = load_dmrg_pbc(L,U) 
		    local xs = lin_factor_pbc(ls,L)
		    local idx = window(L) 
			(Ks_lin[i], as_lin[i]), Ks_lin_err[i], _ = fit_fluc_lin_xs_pbc_err(xs,Fs,L,idx) 
    end    
		local Ls_fit = [5:1:1000;2000;5000;10000; 100000; 1000000;10000000;100000000;1000000000;10000000000;100000000000;100000000000;Inf]
		if i_c == 2 
			# ~sepraratrix 
			local Ks = fss_separatrix(Ls_fit, (Ls_pbc[end],Ks_lin[end])) 
			plot!(p, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		elseif i_c == 1
			# superfluid  
			local Ks = fss_superfluid(Ls_fit, (Ls_pbc[end-1:end], Ks_lin[end-1:end]))
			plot!(p, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		elseif i_c == 3	
			# mott insulator  
			local Ks = fss_mott_insulator(Ls_fit, (Ls_pbc[end-1:end], Ks_lin[end-1:end]))
			plot!(p, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		end 
		
    
        scatter!(p, x_trafo(Ls_pbc), Ks_lin,yerr=Ks_lin_err; marker=:circle, msw=0.6, ms=4, nice_points(c)...)
		scatter!(p, x_trafo(Ls_pbc), Ks_lin ; marker=:circle, msw=2, ms=5, nice_points(c)...)
    
        # OBC 
        local Ks_lin = zeros(Float64,length(Ls_obc))
		local Ks_lin_err = zeros(Float64,length(Ls_obc))
		local as_lin = zeros(Float64,length(Ls_obc))
            
		local factor = 4.1   
		local window_fun_obc = L-> round(Int,L/(2*factor)-L/16):round(Int,L/(2*factor))  
		local error_fun_obc = (xs)->round(Int,2length(xs)÷(8*factor)):round(Int,2length(xs)÷(2factor))
         	
        for (i,L) in enumerate(Ls_obc) 
		    local ls, Fs = load_dmrg_obc(L,U) 
		    local xs = lin_factor_pbc(ls,L)
		    local idx = window_fun_obc(L)
			(Ks_lin[i], as_lin[i]), Ks_lin_err[i], _ = fit_fluc_lin_xs_pbc_err(xs,Fs,L,idx, mask_err=error_fun_obc(xs)) 
end  
        scatter!(p, x_trafo(Ls_obc), Ks_lin,yerr=Ks_lin_err; marker=:circle, msw=0.6, ms=4, m=:square, nice_points(c)...) 
        scatter!(p, x_trafo(Ls_obc), Ks_lin; marker=:circle, msw=2, ms=4, m=:square, nice_points(c)...)

	end
	
	savefig(p,"../figures/002_finite_size_scaling.pdf");
	savefig(p,"../figures/002_finite_size_scaling.svg");
	#p
end 

### Figure 3: Extracting Luttinger Parameter from Flucutations
Extracting the Luttinger parameter $K$ from the particle number fluctuations for different interaction strengths $U/J$. We fit 

\begin{align*}
    \mathcal{F}_{\rm{pbc}}(\ell; K, \alpha) &= 
    \frac{K}{2\pi^2}\ln{\left[1+\frac{\sin^2{(\pi\ell/L)}}{\sinh^2{(\pi\alpha/L)}}\right]}   \approx \frac{K}{\pi^2}\ln\left[\sin\left(\frac{\pi \ell}{L}\right)\right]+A
\end{align*}

to the periodic boundary condition DMRG data (upper panel) for intervals close to $\ell=L/2$, i.e. $\ell\in[L/2-L/16,L/2]$, with the extracted value of $K$ being insensitive to the choice of fitting region. The bottom panel shows fits to the open boundary condition data, where we fit to a small center region $\ell\in[L/8.2-L/16,L/8.2]$ such that $\ell$ covers the same absolute range as in the top panel. This restriction is necessary to mitigate the strong  boundary effects for $\ell$ approaching $L/2$ which leads to a strong dependence of $K$ on the fitting region, ultimately causing a large uncertainty in the extracted value.
 

In [ ]:
begin  
	local Us = [3.2,3.275,3.3]   

    local ylim1=(1.05,1.2)
    local xlim1=(-0.05,0.003)
	local p1 = plot(;ylabel=L"$\mathcal{F}$ (periodic)", bottom_margin=-10mm,ylim=ylim1,xlim=xlim1) # legend(;pos=:bottomright)..., 
	
    # inset
    local insetbox = bbox(0.001,0.002,0.3,0.398)
    pin1 = plot!(p1;
             ylabel=L"\mathcal{F}", 
             inset_subplots=(1, insetbox), 
             subplot=2,
             ymirror = true, 
             xtickfontsize=9, 
             ytickfontsize=9, 
             yticks=([0.6,1.0],["0.6","1.0"]),
             xticks=([-0.3,-0.1]),
             xlim=(-0.37,0.02),
             ylim=(0.3,1.3),
             framestyle=:semi  ) #  framestyle=:semi, :tickfont => (9, :black), xguidefontsize=9, yguidefontsize=9,ylim=(-0.5,7))  
  
    local ylim2=(0.93,1.55)#ylim2=(1.085,1.55)
    local xlim2=(-0.26,0.01)#xlim2=(-0.185,0.01)
    local p2 = plot(;ylabel=L"$\mathcal{F}$ (open)",xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylim=ylim2,xlim=xlim2 , top_margin=-4mm, bottom_margin=-5mm ) # legend(;pos=:bottomright)..., 
    pin2 = plot!(p2;
         ylabel=L"\mathcal{F}", 
         inset_subplots=(1, insetbox), 
         subplot=2,
         ymirror = true, 
         xtickfontsize=9, 
         ytickfontsize=9, 
         yticks=([0.6,1.0,1.4],["0.6","1.0","1.4"]),
         xticks=([-0.3,-0.1]),
         xlim=(-0.48,0.05),
         ylim=(0.4,1.7),
         framestyle=:semi ) #  framestyle=:semi, :tickfont => (9, :black), xguidefontsize=9, yguidefontsize=9,ylim=(-0.5,7))  
       
	local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/")) 
	local cols = [col_lookup[U] for U in Us]

    # legend 
    annotate!(p1, -0.007,1.095+0.015,text(latexstring(f"U/J"),:left,10),subplot=1)

	for (U, (i_c,c)) in zip(Us,enumerate(cols)) 
        # PBC
        
		local L = 96
		local ls, Fs = load_dmrg_pbc(L,U);
		local xs = lin_factor_pbc(ls, L);  
		local idx = window(L)
		local (Kfit, afit), Kerr ,  _ = fit_fluc_lin_xs_pbc_err(xs, Fs, L, idx)   
		 
		scatter!(p1, xs,Fs; marker=:circle, msw=2, ms=4, label=latexstring(f"\\ U={U:3.3f}"), nice_points(c)...)
		 
		plot!(p1, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dash,label=nothing)    
		plot!(p1, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing,linewidth=2)   

		# legend:
		scatter!(p1,[-0.010],[1.095-0.015*(i_c-1)];ms=4,nice_points(c)...)
		annotate!(p1, -0.010,1.095-0.015*(i_c-1),text(latexstring(f"\\ \\ \\ {U:3.3f}"),:left,10),subplot=1)
            
        # inset
        scatter!(pin1, xs,Fs; marker=:circle, msw=2, ms=2, label=latexstring(f"\\ U={U:3.3f}"), nice_points(c)..., subplot=2)
		plot!(pin1, xs[length(xs)÷4:length(xs)], fluctuations_pbc_large_L_xs(xs[length(xs)÷4:length(xs)], (Kfit, afit), (L,)),color=:black,ls=:dash,label=nothing, subplot=2)    
        plot!(pin1, xs[1:end], fluctuations_pbc_large_L_xs(xs[1:end], (Kfit, afit), (L,)),color=:black,ls=:dot,label=nothing, subplot=2)   
		plot!(pin1, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing,linewidth=2, subplot=2)   

        # OBC 
        local factor = 4.1   
		local window_fun_obc = L-> round(Int,L/(2*factor)-L/16):round(Int,L/(2*factor)) 
		local error_fun_obc = (xs)->round(Int,2length(xs)÷(8*factor)):round(Int,2length(xs)÷(2factor))
 
        local L = 460
		local ls, Fs = load_dmrg_obc(L,U);
		local xs = lin_factor_pbc(ls, L);  
		local idx = window_fun_obc(L)
		local (Kfit, afit), Kerr ,  _ = fit_fluc_lin_xs_pbc_err(xs, Fs, L, idx, mask_err=error_fun_obc(xs))   
		 
		scatter!(p2, xs,Fs; marker=:circle, msw=2, ms=4, label=latexstring(f"\\ {U:3.3f}"), nice_points(c)...)
		 
		plot!(p2, xs[error_fun_obc(xs)], fluctuations_pbc_large_L_xs(xs[error_fun_obc(xs)], (Kfit, afit), (L,)),color=:black,ls=:dash,label=nothing)   
        plot!(p2, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dot,label=nothing)    
		plot!(p2, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing,linewidth=2)   
 
        # inset
        scatter!(pin2, xs,Fs; marker=:circle, msw=2, ms=2, label=latexstring(f"\\ {U:3.3f}"), nice_points(c)..., subplot=2)
        plot!(pin2, xs[error_fun_obc(xs)], fluctuations_pbc_large_L_xs(xs[error_fun_obc(xs)], (Kfit, afit), (L,)),color=:black,ls=:dash,label=nothing, subplot=2)   
		plot!(pin2, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dot,label=nothing, subplot=2)    
		plot!(pin2, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing,linewidth=2, subplot=2)   
	end
    
    annotate!(p1,-0.027,1.065,text(L"L=96",10,:left),subplot=1)
    annotate!(p2,-0.143,0.99,text(L"L=460",10,:left),subplot=1)
      
    plot!(pin1, [xlim1[1],xlim1[2],xlim1[2],xlim1[1],xlim1[1]], [ylim1[1],ylim1[1],ylim1[2],ylim1[2],ylim1[1]]; color=:black, subplot=2, lw=0.3, ls=:dot) 
    plot!(pin2, [xlim2[1],xlim2[2],xlim2[2],xlim2[1],xlim2[1]], [ylim2[1],ylim2[1],ylim2[2],ylim2[2],ylim2[1]]; color=:black, subplot=2, lw=0.3, ls=:dot) 
     
	local l = grid(2, 1)
	local p = plot(p1,p2, layout = l, size=(1.2*300,2.2*200), bottom_margin=[0mm -3mm])

	savefig(p,"../figures/003_fit_K_to_fluctuations.pdf");
	savefig(p,"../figures/003_fit_K_to_fluctuations.svg");
	#p
end

#### Figure 4: Extracting Crtical Point
Dependence of the flow parameter $\zeta$ on the interaction strength $U/J$ (ground truth, red circles).  A Gaussian process (GP) is used to perform a non-parametric fit to the data, with the predicted posterior mean shown as a solid red line, accompanied by a red shaded region indicating the posterior confidence interval (CI).  The critical point occurs when $\zeta=1$ yielding $U_c/J=3.276\pm0.002$ indicated by the vertical blue line, and a shaded region of uncertainty in the inset.  Near the transition, a linear fit to $U/J\in\{3.300, 3.290, 3.280, 3.275\}$ (green dashed line) yields the same critical point and uncertainty, but away it deviates outside of error bars.

See other ipynb using Python code to generate this figure.

## Supplementary Material

### Figure S1: Difference between open and periodic boundary conditions

Comparison of particle number fluctuations for Bose-Hubbard chains with periodic (top left panel) and open boundary conditions (top right panel). We show the fluctuations $\mathcal{F}$ for partition sizes $\ell $ up to  $L/2$ for several values of the interaction strength $U/J$ with linear fits (solid black) over regions described in the main text and extended (dashed black). The bottom panel shows the flow of $K$ depending on the system size $L$, with a good agreement between PBC (circles) and OBC results (squares), owing to the fine-tuned OBC fitting method illustrated in Fig. S2.

In [ ]:
begin 
    local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/")) 

    local p_pbc = plot(;axis=:off,size=(1.2*200,1.2*160),xlim=(-1.6,1.2),ylim=(-1.25,1.4), bottom_margin=[-20mm -20mm] , top_margin=[2mm 2mm] )
    draw_periodic_lattice!(p_pbc ; n=13, nice_points("#000000")... , mswidth  =1 )
    
    local p_obc = plot(;axis=:off,size=(1.2*300,1.2*160), ylim=(-0.5,0.5),xlim=(-1,10.5), bottom_margin=[-20mm -20mm])
    draw_open_lattice!(p_obc; nice_points("#000000")... , mswidth  =1 ) 

    Us = [3.0,3.1,3.2,3.3]
    # fluctuations 
    # pbc 
    local p_F_pbc = plot(;xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylabel=L"$\mathcal{F}$ (periodic)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm], xlim=(-0.2,0.02) ,ylim=(0.7,1.3))
    plot_fluctuations_supplement!(p_F_pbc, Us, 96, col_lookup; load_fun=load_dmrg_pbc)
    annotate!(p_F_pbc,-0.18,1.25,text(latexstring("L=96"),11,:left))

    # obc 
    local factor = 4.1  
    local window_fun_obc = L-> round(Int,L/(2*factor)-L/16):round(Int,L/(2*factor)) 
    local error_fun_obc = (xs)->round(Int,2length(xs)÷(8*factor)):round(Int,2length(xs)÷(2factor))

    local p_F_obc = plot(;xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylabel=L"$\mathcal{F}$ (open)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(-0.3,0.02) ,ylim=(0.82,1.68), xticks=[-0.3,-0.2,-0.1,-0.0])
    plot_fluctuations_supplement!(p_F_obc, Us, 460, col_lookup; load_fun=load_dmrg_obc, legend=false,window_fun=window_fun_obc, first_point=1,first_fit_value=30,mask_err=error_fun_obc)
    annotate!(p_F_obc,-0.18,1.6,text(latexstring("L=460"),11,:left))

    
    # K(L)
    x_trafo = (x) -> 1.0 ./ log.(x)
    # pbc 
    local p_K = plot(;xlabel=L"$1/\ln(L)$",ylabel=L"$K$",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(0.14,0.4) )
    plot_K_supplement!(p_K, Us, col_lookup; load_fun=load_dmrg_pbc, L_lookup=U->get_values_of_L("../data/pbc/DMRG/",U),marker=:circle )
    # obc  
    plot_K_supplement!(p_K, Us, col_lookup; load_fun=load_dmrg_obc ,L_lookup=U->get_values_of_L_obc("../data/obc/DMRG/",U),window_fun=window_fun_obc,mask_err=error_fun_obc,marker=:square,ms=4)

    # combine plots   
    local l = @layout [ 
        a{.4h}  b{.4h} 
        c{1.2h} d{1.2h} 
        a{0.41h}            
        ]
    println(l)
    local p = plot(p_obc,p_pbc, p_F_obc,p_F_pbc, p_K,layout=l, size=2.0.*(1.0*315,1.0*325*0.9),margin = 0mm,           
    bottom_margin = [-28mm -28mm  6mm 6mm 0mm 0mm],   
    top_margin = -1mm,
    spacing = 0.0   )        

    savefig(p,"../figures/S001_compare_pbc_obc.pdf");
    savefig(p,"../figures/S001_compare_pbc_obc.svg");
end

### Figure S2: Fitting methods OBC 

Extracting the Luttinger parameter $K$ from the particle number fluctuations for different fit intervals to the DMRG data for open boundary conditions (OBC). We fit to the OBC DMRG data at $L=460$ (upper panels) for intervals $\ell \in [\ell_{\rm max}-L/16,\ell_{\rm max}]$ with $\ell_{\rm max}=L/6$ (left), $\ell_{\rm max}=L/8.2$ (center),  $\ell_{\rm max}=L/10$ (right). The lower panels depict the corresponding finite size scaling of the Luttinger parameter. Analogous to main text Fig. 2, we show RG flows from the superfluid phase, separatrix, and insulating phase. The gray points are the results for periodic boudaries as presented in the main text. Squares show the open boundary values of $K$ obtained with the corresponding fit intervals. In contrast to the periodic case, we find strong dependence of the Luttinger parameters from the fitting interval for open boundaries. In the main text, we use the fine-tuned fit interval depicted in the center panels.

In [ ]:
begin
    ################### Fit window 1 ###################
    # window definition
    local l_max = L -> L/6 
    local fit_window_obc =  L-> round(Int,l_max(L)-L/16):round(Int,l_max(L)) 
    local err_window_obc =  (xs)->round(Int,l_max(2length(xs))/4):round(Int,l_max(2length(xs)))
    # panelW
    local p11 = plot(;xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylabel=L"$\mathcal{F}$ (open)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(-0.55,0.07) ,ylim=(0.35,1.6), xticks=[-0.4,-0.2,-0.0],title=L"\ell_{\mathrm{max}}=L/6",yticks=[0.5,0.75,1.0,1.25,1.5]) 
    local p21 = plot(;ylabel=L"$K$",xlabel=L"$1/\ln(L)$",xlim=(0.0,0.375),ylim=(1.99,2.27), size=(1.2*300,1.0*250),bottom_margin=[0mm 0mm 0mm], yticks=[2.0,2.05,2.1,2.15,2.2,2.25])  

    # plot
    local _,K,Kerr=plot_fig3_Fandfit_obc!(p11, fit_window_obc, err_window_obc)
    annotate!(p11,0.0,0.5,text(latexstring(f"$K={K:2.2f}\\pm{Kerr:2.2f}$"),10,:right))
    plot_fig2_Kobc!(p21,  fit_window_obc, err_window_obc; legend=false)

    ################### Fit window 2 ###################
    # window definition
    local l_max = L -> L/8.2 
    local fit_window_obc =  L-> round(Int,l_max(L)-L/16):round(Int,l_max(L)) 
    local err_window_obc =  (xs)->round(Int,l_max(2length(xs))/4):round(Int,l_max(2length(xs)))
    # panel
    local p12 = plot(;xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$" ,size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(-0.55,0.07) ,ylim=(0.35,1.6),yticks=([0.5,0.75,1.0,1.25,1.5],["","","","",""]), xticks=[-0.4,-0.2,-0.0],title=L"\ell_{\mathrm{max}}=L/8.2") 
    local p22 = plot(; xlabel=L"$1/\ln(L)$",xlim=(0.0,0.375),ylim=(1.99,2.27), yticks=([2.0,2.05,2.1,2.15,2.2,2.25],["","","","","",""]), size=(1.2*300,1.0*250),bottom_margin=[0mm 0mm 0mm]) #,yticks=([1.6,1.8,2.0],["1.60","1.80","2.00"]))

    # plot
    local _,K,Kerr=plot_fig3_Fandfit_obc!(p12, fit_window_obc, err_window_obc)
    annotate!(p12,0.0,0.5,text(latexstring(f"$K={K:2.2f}\\pm{Kerr:2.2f}$"),10,:right))
    plot_fig2_Kobc!(p22,  fit_window_obc, err_window_obc; legend=true)
 
    
    ################### Fit window 3 ###################
    # window definition
    local l_max = L -> L/10 
    local fit_window_obc =  L-> round(Int,l_max(L)-L/16):round(Int,l_max(L)) 
    local err_window_obc =  (xs)->round(Int,l_max(2length(xs))/4):round(Int,l_max(2length(xs)))
    # panel
    local p13 = plot(;xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$" ,size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(-0.55,0.07) ,ylim=(0.35,1.6),yticks=([0.5,0.75,1.0,1.25,1.5],["","","","",""]), xticks=[-0.4,-0.2,-0.0],title=L"\ell_{\mathrm{max}}=L/10") 
    local p23 = plot(; xlabel=L"$1/\ln(L)$",xlim=(0.0,0.375),ylim=(1.99,2.27), yticks=([2.0,2.05,2.1,2.15,2.2,2.25],["","","","","",""]), size=(1.2*300,1.0*250),bottom_margin=[0mm 0mm 0mm]) #,yticks=([1.6,1.8,2.0],["1.60","1.80","2.00"]))

    # plot
    local _,K,Kerr=plot_fig3_Fandfit_obc!(p13, fit_window_obc, err_window_obc)
    annotate!(p13,0.0,0.5,text(latexstring(f"$K={K:2.2f}\\pm{Kerr:2.2f}$"),10,:right))
    plot_fig2_Kobc!(p23,  fit_window_obc, err_window_obc; legend=false)
  

    ### Combine plots
    local l = grid(2, 3; heights=[0.4,0.6], widths=[1/3,1/3,1/3 ]) 
    local p = plot(p11, p12, p13, p21, p22, p23,layout=l, size=2.0.*(1.0*315,1.0*325*0.7), bottom_margin=[-0mm -0mm -0mm 0mm 0mm 0mm], top_margin = [0mm 0mm 0mm 0mm 0mm 0mm], left_margin=[2mm -5mm -5mm 2mm -5mm -5mm])

    savefig(p,"../figures/S003_obc_fit_interval_data.pdf");
    savefig(p,"../figures/S003_obc_fit_interval_data.svg");
end

###  Figure S3: Reproduce literature result

Reproduction of the fitting method (left panel) and our inferred finite size scaling (right panel) of Ref. [S6] for open boundary condition fluctuations in region $A = [0,\ell]$. The left panel depicts the dependence of the extracted Luttinger parameter when fitting to intervals $\ell\in[4,w+4]$ (red squares), $[16,16+w]$ (green diamonds), and $[L/2-w/2,L/2+w/2]$ (black circles) on the size $w$ of the fitting window for $L=128, U/J=3.3333$. The Luttinger parameter is then estimated as the mean between center and left $(r=4)$ interval at $L/w=8$. Our left panel is in excellent agreement with the inset in the right panel of Fig. 2 in the supplement of Ref. [S6]. The right panel shows finite size scaling of the Luttinger parameter for the points of Ref. [S6] (red) and more system sizes in between (blue) at the critial point $U/J=3.3456$ reported in Ref. [S6]. While scaling as $1/\ln(L)$ to $L\to\infty$ only from the red points yields a $K(L\to\infty)\approx 2$ within error bars (black circle),  the finite size scaling including many system sizes is inconsistent with extrapolation from the red points only. This inconsistency of extrapolation is reflected by the large error bars we obtain for the red points. We attribute these observations to the strong boundary effects fully present in the $A=[0,\ell]$ region for open boundary conditions.

[S6] S. Rachel, N. Laflorencie, H. F. Song, and K. Le Hur, Physical Review Letters 108, 116401 (2012)

In [ ]:
begin
    local L = 128
    local U = 3.33333
    
    # lattice schematic
    local p_obc = plot(;axis=:off,size=(1.2*160,1.2*300), ylim=(-0.5,10.5),xlim=(-0.5,0.5),left_margin=-11mm,right_margin=-4mm,bottom_margin=-15mm)
    draw_open_lattice_horizontal!(p_obc; nice_points("#000000")... , mswidth  =1 )  
    
    # fitting method
    local p_fit_obc = plot(;xlabel=L"$L/w$",ylabel=L"$K$ (open)",size=(1.2*315,1.2*225), xlim=(.5,8.5) ,ylim=(1.90,2.07),title=L"U/J=3.3333" )
    Kfit,Kerr,_ = plot_fitting_method_literature!(p_fit_obc, U, L;colors=["#000000","#377a30","#d62728"])
    annotate!(2.5,1.98,text(latexstring(f"K={Kfit:3.2f}\\pm{Kerr:3.2f}"),10,:left))
    annotate!(2,1.945,text(L"\mathrm{center}\ \mathrm{window} ",10,:left))
        scatter!(p_fit_obc,[1.6],[1.944];m=:circle,nice_points("#000000")...)
    annotate!(2,1.93,text(L"$\mathrm{left}\ \mathrm{window}\ r=16$",10,:left))
        scatter!(p_fit_obc,[1.6],[1.93];m=:diamond,nice_points("#377a30")...)
    annotate!(2,1.915,text(L"$\mathrm{left}\ \mathrm{window}\ r=4$",10,:left))
        scatter!(p_fit_obc,[1.6],[1.915];m=:square,nice_points("#d62728")...) 
    
    # finite size scaling in literature
    local U = 3.3456
    local p_fss_obc = plot(;xlabel=L"$1/\ln L$",ylabel=L"$K$ (open)",size=(1.2*315,1.2*225), xlim=(-0.01,0.299) ,ylim=(1.98,2.042),title=L"U/J=3.3456")
    local Ls_lit = [128,256,512] 
    local Ls_more = [64,96,180,200,220,240,260,280,330,380,420,460] 
    plot_fss_method_literature!(p_fss_obc, U, Ls_lit, Ls_more; colors=["#779ECB","#FF6961"])
    
    # combine plots 
    widhts = [0.2,1,1]
    local l = grid(1, 3; widths= widhts / sum(widhts))
    
    local p = plot(p_obc,p_fit_obc,p_fss_obc,layout=l, size= 2.0.*(1.0*315,1.0*325/2.7), bottom_margin=[-10mm -10mm 2mm ], top_margin = [0mm 0mm 0mm])

    savefig(p,"../figures/S002_comparison_obc_literature.pdf"); 
end